In [4]:
import os
import torch
from torch_robotics.torch_utils.seed import fix_random_seed
from torch_robotics.torch_utils.torch_utils import get_torch_device
from mmd.trainer import get_dataset
from mmd.utils.loading import load_params_from_yaml


seed = 18 #params.seed
fix_random_seed(seed)

TRAINED_MODELS_DIR = '../../data_trained_models/'
trained_models_dir = TRAINED_MODELS_DIR
model_id = 'EnvEmptyNoWait2D-RobotPlanarDisk'
model_dir = os.path.join(trained_models_dir, model_id)
results_dir = os.path.join(model_dir, 'results_inference', str(seed))
os.makedirs(results_dir, exist_ok=True)
args = load_params_from_yaml(os.path.join(model_dir, "args.yaml"))

device='cuda' #params.device
device = get_torch_device(device)
tensor_args = {'device': device, 'dtype': torch.float32}

train_subset, train_dataloader, val_subset, val_dataloader = get_dataset(
    dataset_class='TrajectoryDataset',
    use_extra_objects=True,
    obstacle_cutoff_margin=0.05,
    **args,
    tensor_args=tensor_args
)
dataset = train_subset.dataset



---------------Loading data
Precomputing the SDF grid and gradients took: 0.273 sec
TrajectoryDataset
n_trajs: 10000
trajectory_dim: (64, 4)



# Init Everything

## Everything befroe go into main

In [1]:
import os
from datetime import datetime
import time

import torch
from einops._torch_specific import allow_ops_in_compiled_graph  # requires einops>=0.6.1
from typing import Tuple, List

from torch_robotics.robots import *
from torch_robotics.torch_utils.torch_utils import get_torch_device
# from mmd.planners.multi_agent import CBS, PrioritizedPlanning
from mmd.planners.multi_agent import End2EndPlanning
from mmd.planners.single_agent import MPD, MPDEnd2End, MPDEnsemble
from mmd.common.constraints import MultiPointConstraint
from mmd.common.conflicts import PointConflict
from mmd.common.trajectory_utils import densify_trajs
from mmd.common import get_start_goal_pos_circle
from mmd.common.pretty_print import *
from mmd.config.mmd_params import MMDParams as params
from mmd.common.experiments import MultiAgentPlanningSingleTrialConfig, MultiAgentPlanningSingleTrialResult, \
    get_result_dir_from_trial_config, TrialSuccessStatus

allow_ops_in_compiled_graph()
TRAINED_MODELS_DIR = '../../data_trained_models/'
device = 'cuda'
device = get_torch_device(device)
tensor_args = {'device': device, 'dtype': torch.float32}

test_config_single_tile = MultiAgentPlanningSingleTrialConfig()
test_config_single_tile.num_agents = 3
test_config_single_tile.instance_name = "test"
# test_config_single_tile.multi_agent_planner_class = "XECBS" 
test_config_single_tile.single_agent_planner_class = "MPDe2e" # Or "MPDEnsemble"
test_config_single_tile.stagger_start_time_dt = 0
test_config_single_tile.runtime_limit = 60 * 3  # 3 minutes.
test_config_single_tile.time_str = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
test_config_single_tile.render_animation = True

example_type = "single_tile"
test_config_single_tile.global_model_ids = [['EnvEmptyNoWait2D-RobotPlanarDisk']]
# test_config_single_tile.global_model_ids = [['EnvConveyor2D-RobotPlanarDisk']]
# test_config_single_tile.global_model_ids = [['EnvHighways2D-RobotPlanarDisk']]
# test_config_single_tile.global_model_ids = [['EnvDropRegion2D-RobotPlanarDisk']]

# Choose starts and goals.
test_config_single_tile.agent_skeleton_l = [[[0, 0]]] * test_config_single_tile.num_agents
torch.random.manual_seed(10)
# start & goal are uniformly distributed on a circle with radiu 0.8
test_config_single_tile.start_state_pos_l, test_config_single_tile.goal_state_pos_l = \
get_start_goal_pos_circle(test_config_single_tile.num_agents, 0.8)
print("Starts:", test_config_single_tile.start_state_pos_l)
print("Goals:", test_config_single_tile.goal_state_pos_l)
# test_config_single_tile.n_diffusion_steps = 50

#run_multi_agent_trial(test_config_single_tile)
#print(GREEN, 'OK.', RESET)

/local-scratch/localhome/lya108/mmd/mmd/trainer/trainer.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Starts: [tensor([0.8000, 0.0000], device='cuda:0'), tensor([-0.4000,  0.6928], device='cuda:0'), tensor([-0.4000, -0.6928], device='cuda:0')]
Goals: [tensor([-8.0000e-01,  9.7972e-17], device='cuda:0'), tensor([ 0.4000, -0.6928], device='cuda:0'), tensor([0.4000, 0.6928], device='cuda:0')]


## main, before planning

In [2]:
test_config = test_config_single_tile
start_time_l = [i * test_config.stagger_start_time_dt for i in range(test_config.num_agents)]
print(f'start_time_l:{start_time_l}')

# ============================
# Arguments for the single diffusion planner.
# diffusion model & single agent planner
# ============================
diffusion_planner_model_args = {
    # config for diffusion model - no change
    'planner_alg': 'mmd',
    'use_guide_on_extra_objects_only': params.use_guide_on_extra_objects_only,  #False
    'n_samples': params.n_samples,  # how many trajs to be generated by one diffusion model (64)
    'n_local_inference_noising_steps': params.n_local_inference_noising_steps,  # 3
    'n_local_inference_denoising_steps': params.n_local_inference_denoising_steps,
    'start_guide_steps_fraction': params.start_guide_steps_fraction,
    'n_guide_steps': params.n_guide_steps,
    'n_diffusion_steps_without_noise': params.n_diffusion_steps_without_noise,
    # config for low-level planner
    'weight_grad_cost_collision': params.weight_grad_cost_collision,
    'weight_grad_cost_smoothness': params.weight_grad_cost_smoothness,
    'weight_grad_cost_constraints': params.weight_grad_cost_constraints,
    'weight_grad_cost_soft_constraints': params.weight_grad_cost_soft_constraints,
    'factor_num_interpolated_points_for_collision': params.factor_num_interpolated_points_for_collision,
    'trajectory_duration': params.trajectory_duration,
    'device': params.device,
    'debug': params.debug,
    'seed': params.seed,
    'results_dir': params.results_dir,
    'trained_models_dir': TRAINED_MODELS_DIR,
}
end_to_end_planner_model_args = {
    'start_time_l': start_time_l,
    'runtime_limit': test_config.runtime_limit,
    # 'n_diffusion_steps': test_config.n_diffusion_steps,
    'conflict_type_to_constraint_types': {PointConflict: {MultiPointConstraint}},
    'device': params.device,
    # device, seed, debug需要挪出来吗？
}

# ============================
# Create a results directory.
# ============================
results_dir = get_result_dir_from_trial_config(test_config, test_config.time_str, test_config.trial_number)
os.makedirs(results_dir, exist_ok=True)
num_agents = test_config.num_agents

# ============================
# Get planning problem.
# ============================
# If want to get random starts and goals, then must do that after creating the reference task and robot.
start_l = test_config.start_state_pos_l
goal_l = test_config.goal_state_pos_l
global_model_ids = test_config.global_model_ids
agent_skeleton_l = test_config.agent_skeleton_l
print(f'debug: global_model_ids: {global_model_ids}')

# ============================
# Transforms and model tiles setup.
# ============================
# Create a reference planner from which we'll use the task and robot as the reference on in CBS.
# Those are used for collision checking and visualization. This has a skeleton of all tiles.
reference_agent_skeleton = [[r, c] for r in range(len(global_model_ids))
                            for c in range(len(global_model_ids[0]))]

# ============================
# Transforms from tiles to global frame.
# ============================
tile_width = 2.0
tile_height = 2.0
global_model_transforms = [[torch.tensor([x * tile_width, -y * tile_height], **tensor_args)
                            for x in range(len(global_model_ids[0]))] for y in range(len(global_model_ids))]    
print(f'debug: global model tansforms:{global_model_transforms}')

# ============================
# Parse the single agent planner class name.
# emmm in this case maybe i should start by parser the model?
# what exactly was done in the low-level planner
# ============================    
if test_config.single_agent_planner_class == "MPDe2e":
    planner_class = MPDEnd2End
elif test_config.single_agent_planner_class == "MPD":
    planner_class = MPD
elif test_config.single_agent_planner_class == "MPDEnsemble":
    planner_class = MPDEnsemble
else:
    raise ValueError(f'Unknown single agent planner class: {test_config.single_agent_planner_class}')

# ============================
# Create reference agent planner.
# ============================
# And for the reference skeleton.
reference_task = None
reference_robot = None
reference_agent_transforms = {}
reference_agent_model_ids = {}

for skeleton_step in range(len(reference_agent_skeleton)):
    skeleton_model_coord = reference_agent_skeleton[skeleton_step]
    reference_agent_transforms[skeleton_step] = global_model_transforms[skeleton_model_coord[0]][
        skeleton_model_coord[1]]
    reference_agent_model_ids[skeleton_step] = global_model_ids[skeleton_model_coord[0]][
        skeleton_model_coord[1]]
reference_agent_model_ids = [reference_agent_model_ids[i] for i in range(len(reference_agent_model_ids))]
# Create the reference low level planner
diffusion_planner_model_args['start_state_pos'] = torch.tensor([0.5, 0.9], **tensor_args)  # This does not matter.
diffusion_planner_model_args['goal_state_pos'] = torch.tensor([-0.5, 0.9], **tensor_args)  # This does not matter.
diffusion_planner_model_args['model_ids'] = reference_agent_model_ids  # This matters.
diffusion_planner_model_args['transforms'] = reference_agent_transforms  # This matters.

if test_config.single_agent_planner_class in ["MPD", "MPDe2e"]:
    diffusion_planner_model_args['model_id'] = reference_agent_model_ids[0]

reference_single_agent_planner = planner_class(**diffusion_planner_model_args)
reference_task = reference_single_agent_planner.task
reference_robot = reference_single_agent_planner.robot


start_time_l:[0, 0, 0]
debug: global_model_ids: [['EnvEmptyNoWait2D-RobotPlanarDisk']]
debug: global model tansforms:[[tensor([0., 0.], device='cuda:0')]]
####################################
Initializing Planner with Model -- EnvEmptyNoWait2D-RobotPlanarDisk
Algorithm -- mmd

---------------Loading data
Precomputing the SDF grid and gradients took: 0.345 sec


/local-scratch/localhome/lya108/mmd/mmd/datasets/trajectories.py:91: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  trajs_free_tmp = torch.load(


TrajectoryDataset
n_trajs: 10000
trajectory_dim: (64, 4)

[ models/temporal ] Channel dimensions: [(4, 32), (32, 64), (64, 128)]


/local-scratch/localhome/lya108/mmd/mmd/planners/single_agent/mpd_inner.py:129: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load(os.path.join(model_dir, 'checkpoints

start_state_pos: tensor([0.5000, 0.9000], device='cuda:0')
goal_state_pos: tensor([-0.5000,  0.9000], device='cuda:0')
start_state_pos: tensor([0.5000, 0.9000], device='cuda:0')
goal_state_pos: tensor([-0.5000,  0.9000], device='cuda:0')
debug: get hard_conds via dataset.get_hard_conditions: {0: tensor([5.1285e-01, 9.2320e-01, 4.4692e-04, 5.0635e-03], device='cuda:0'), 63: tensor([-5.1295e-01,  9.2320e-01,  4.4692e-04,  5.0635e-03], device='cuda:0')}
debug: prepare to constract cost_collision_list
debug: cost_composite list got
debug: guide is formulated


In [3]:
# ============================
# Run trial.
# ============================
exp_name = f'mmd_single_trial'

# Transform starts and goals to the global frame. Right now they are in the local tile frames.
start_l = [start_l[i] + global_model_transforms[agent_skeleton_l[i][0][0]][agent_skeleton_l[i][0][1]]
            for i in range(num_agents)]
goal_l = [goal_l[i] + global_model_transforms[agent_skeleton_l[i][-1][0]][agent_skeleton_l[i][-1][1]]
            for i in range(num_agents)]

# ============================
# Create global transforms for each agent's skeleton.
# ============================
# Each agent has a dict entry. Each entry is a dict with the skeleton steps (0, 1, 2, ...), mapping to the
# model transform.
agent_model_transforms_l = []
agent_model_ids_l = []
for agent_id in range(num_agents):
    agent_model_transforms = {}
    agent_model_ids = {}
    for skeleton_step in range(len(agent_skeleton_l[agent_id])):
        skeleton_model_coord = agent_skeleton_l[agent_id][skeleton_step]
        agent_model_transforms[skeleton_step] = global_model_transforms[skeleton_model_coord[0]][
            skeleton_model_coord[1]]
        agent_model_ids[skeleton_step] = global_model_ids[skeleton_model_coord[0]][skeleton_model_coord[1]]
    agent_model_transforms_l.append(agent_model_transforms)
    agent_model_ids_l.append(agent_model_ids)
# Change the dict of the model ids to a list sorted by the skeleton steps.
agent_model_ids_l = [[agent_model_ids_l[i][j] for j in range(len(agent_model_ids_l[i]))] for i in
                        range(num_agents)]
# ============================
# Create the single agent planners.
# ============================
planners_creation_start_time = time.time()
single_agent_planner_l = []
for i in range(num_agents):
    single_agent_planner_model_args_i = diffusion_planner_model_args.copy()
    single_agent_planner_model_args_i["start_state_pos"] = start_l[i]
    single_agent_planner_model_args_i['goal_state_pos'] = goal_l[i]
    single_agent_planner_model_args_i['model_ids'] = agent_model_ids_l[i]
    single_agent_planner_model_args_i["transforms"] = agent_model_transforms_l[i]
    if test_config.single_agent_planner_class in ["MPD", "MPDe2e"]:
        single_agent_planner_model_args_i["model_id"] = agent_model_ids_l[i][0]
    single_agent_planner_l.append(planner_class(**single_agent_planner_model_args_i))
print('Planners creation time:', time.time() - planners_creation_start_time)
print("\n\n\n\n")

# ============================
# Create the multi agent planner.
# don't really exist, but we use sth like PP
# ============================
planner = End2EndPlanning(single_agent_planner_l,
                            start_l,
                            goal_l,
                            reference_task=reference_task,
                            reference_robot=reference_robot,
                            **end_to_end_planner_model_args)

####################################
Initializing Planner with Model -- EnvEmptyNoWait2D-RobotPlanarDisk
Algorithm -- mmd

---------------Loading data
Precomputing the SDF grid and gradients took: 0.270 sec
TrajectoryDataset
n_trajs: 10000
trajectory_dim: (64, 4)

[ models/temporal ] Channel dimensions: [(4, 32), (32, 64), (64, 128)]
start_state_pos: tensor([0.8000, 0.0000], device='cuda:0')
goal_state_pos: tensor([-8.0000e-01,  9.7972e-17], device='cuda:0')
start_state_pos: tensor([0.8000, 0.0000], device='cuda:0')
goal_state_pos: tensor([-8.0000e-01,  9.7972e-17], device='cuda:0')
debug: get hard_conds via dataset.get_hard_conditions: {0: tensor([8.2059e-01, 5.5552e-05, 4.4692e-04, 5.0635e-03], device='cuda:0'), 63: tensor([-8.2068e-01,  5.5552e-05,  4.4692e-04,  5.0635e-03], device='cuda:0')}
debug: prepare to constract cost_collision_list
debug: cost_composite list got
debug: guide is formulated
####################################
Initializing Planner with Model -- EnvEmptyNoWait2

# Planning

In [5]:
import os
import time
from copy import copy
import einops
from pathlib import Path
import matplotlib.pyplot as plt
import torch
from typing import Tuple, List

from torch_robotics.trajectory.metrics import compute_smoothness, compute_path_length, compute_variance_waypoints
from mp_baselines.planners.costs.cost_functions import CostCollision, CostComposite, CostConstraint
from torch_robotics.torch_utils.torch_timer import TimerCUDA
from mmd.common.conflicts import Conflict
from mmd.common.constraints import MultiPointConstraint
from mmd.common.experiments import TrialSuccessStatus
from mmd.common.pretty_print import *
from mmd.config import MMDParams as params
from mmd.common import smooth_trajs, is_multi_agent_start_goal_states_valid, global_pad_paths
from mmd.models.diffusion_models.sample_functions import apply_hard_conditioning, ddpm_sample_fn

In [6]:
start_time = time.time()
num_agent = len(planner.single_agent_planner_l)
device = planner.device
batch_size = params.n_samples
shape = (planner.num_agents, batch_size, params.horizon, planner.single_agent_planner_l[0].state_dim)
# added
warm_start_path_b = None
t_start_guide = planner.single_agent_planner_l[0].t_start_guide
n_diffusion_steps = planner.single_agent_planner_l[0].model.n_diffusion_steps
return_chain = True
n_diffusion_steps_without_noise=planner.single_agent_planner_l[0].n_diffusion_steps_without_noise

if warm_start_path_b is not None:
    x = warm_start_path_b
    print(CYAN, "Using warm start path in p_sample_loop. Steps (negative attempts to recon. t=0)", [n_diffusion_steps, -n_diffusion_steps_without_noise], RESET)
else:
    x = torch.randn(shape, device=device)
    print(CYAN, "Using random noise in p_sample_loop. Steps (negative attempts to recon. t=0)", [n_diffusion_steps, -n_diffusion_steps_without_noise], RESET)
#if 't_start_guide' in sample_kwargs:
#    print(CYAN, "Starting to guide after t <", sample_kwargs['t_start_guide'], RESET)
print(CYAN, "Starting to guide after t <", t_start_guide, RESET)
# x = apply_hard_conditioning(x, agent.hard_conds)
chain = [x] if return_chain else None



 Using random noise in p_sample_loop. Steps (negative attempts to recon. t=0) [25, -1] 
 Starting to guide after t < 13 


In [7]:
prev_step_trajs = x
constraint_l = [[] for _ in range(num_agent)]

In [8]:
def make_timesteps(batch_size, i ,device):
    t = torch.full((batch_size,), i, device=device, dtype=torch.long)
    return t
i = 24
t= make_timesteps(batch_size, i, device)

In [9]:
j = 0
agent = planner.single_agent_planner_l[j]

In [10]:
from copy import copy
hard_conds = copy(agent.hard_conds)
context = copy(agent.context)
for k, v in hard_conds.items():
    new_state = einops.repeat(v, 'd -> b d', b=batch_size)
    hard_conds[k] = new_state

if context is not None:
    for k, v in context.items():
        context[k] = einops.repeat(v, 'd -> b d', b=batch_size)

In [13]:
x[j] = apply_hard_conditioning(x[j], hard_conds)

In [14]:
constraint_l[j] = planner.create_soft_constraints_from_other_agents_paths(prev_step_trajs, agent_id=j)

In [15]:
agent.update_constraints(constraint_l[j])

In [21]:
x[j], _ = ddpm_sample_fn(agent.model, x[j], hard_conds, context, t, guide=agent.guide)

> /local-scratch/localhome/lya108/mmd/mmd/models/diffusion_models/sample_functions.py(58)ddpm_sample_fn()
     56     import pdb; pdb.set_trace()
     57 
---> 58     model_mean, _, model_log_variance = model.p_mean_variance(x=x, hard_conds=hard_conds, context=context, t=t)
     59     x = model_mean
     60 

<class 'torch.Tensor'>
torch.Size([64, 64, 4])
torch.Size([64])
> /local-scratch/localhome/lya108/mmd/mmd/models/diffusion_models/diffusion_model_base.py(153)p_mean_variance()
    151 
    152         import pdb; pdb.set_trace()
--> 153         x_recon = self.predict_start_from_noise(x, t=t, noise=self.model(x, t, context))
    154 
    155         if self.clip_denoised:

torch.Size([64, 64, 4])


In [22]:
x[j] = apply_hard_conditioning(x[j], hard_conds)

In [ ]:
agent.guide.reset_extra_costs()

In [16]:
t_single = t[0]

In [17]:
if t_single <0:
    print('t_single < 0')
    t = torch.zeros_like(t)

In [18]:
def extract(a, t, x_shape):
    b, *_ = t.shape
    out = a.gather(-1, t)
    return out.reshape(b, *((1,) * (len(x_shape) - 1)))

In [19]:
x_t = x
x_t

tensor([[[[ 8.2059e-01,  5.5552e-05,  4.4692e-04,  5.0635e-03],
          [-2.3162e+00,  5.2466e-01,  4.7068e-02,  9.4156e-02],
          [-6.5604e-01,  4.4546e-01, -5.2948e-02,  8.5606e-01],
          ...,
          [ 1.1000e+00,  2.5674e-02,  6.7181e-01,  1.3306e+00],
          [ 5.4286e-01,  2.2151e-01, -2.2344e-01,  1.2278e+00],
          [-8.2068e-01,  5.5552e-05,  4.4692e-04,  5.0635e-03]],

         [[ 8.2059e-01,  5.5552e-05,  4.4692e-04,  5.0635e-03],
          [-9.2886e-01, -5.8093e-01, -2.3160e-01,  3.1880e-01],
          [-1.2408e-01,  3.6317e-01,  1.4351e+00,  3.5971e-02],
          ...,
          [ 3.1652e-01,  3.4372e-01,  6.2235e-01,  3.2994e-01],
          [ 2.1875e-03, -7.7053e-01,  1.0030e+00, -9.2270e-02],
          [-8.2068e-01,  5.5552e-05,  4.4692e-04,  5.0635e-03]],

         [[ 8.2059e-01,  5.5552e-05,  4.4692e-04,  5.0635e-03],
          [-1.2344e-01, -1.0227e-01, -1.2624e+00, -7.4362e-01],
          [ 5.1342e-01, -2.1779e-01, -4.2178e-01, -8.4532e-01],
      

In [20]:
x_t.shape

torch.Size([3, 64, 64, 4])

In [22]:
noise = agent.model(x_t, t, context)

NotImplementedError: 

In [21]:
extract(agent.model.sqrt_recip_alphas_cumprod, t, x_t.shape) * x_t - extract(agent.model.sqrt_recipm1_alphas_cumprod, t, x_t.shape) * noise

tensor([[[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


        [[[4602.6675]]],


 

In [22]:
len(agent.model.sqrt_recip_alphas_cumprod)

25

In [ ]:
extract(agent.sqrt_recip_alphas_cumprod, t, x_t.shape)

In [ ]:
# model_mean, _, model_log_variance = model.p_mean_variance(x=x, hard_conds=hard_conds, context=context, t=t)
x_recon = agent.model.predict_start_from_noise(x, t=t, noise=agent.model.model(x, t, context))


In [18]:
t.shape

torch.Size([64])

In [17]:
x.shape

torch.Size([3, 64, 64, 4])